In [88]:
import pandas as pd
import csv
import json
import nltk
import numpy as np
from pandas.io.json import json_normalize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import cross_validation
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

file = "C:\\Users\\user\\Desktop\\semantics.json"
file1 = "C:\\Users\\user\\Desktop\\reviews1.json"  #reviews1.json
df = pd.read_json(file1,typ='series')
df_rev = json_normalize(df['Reviews'])
df_rev['full_content'] = df_rev['Content'] + '. ' + df_rev['Title']
pos = pd.read_json(file,typ='series')

newcol = df_rev['full_content']
newcol.to_csv('content_reviews.csv',header=False, index=False, encoding='utf-8')

df = pd.read_csv('C:\Users\user\Documents\sentiment-analysis-reviews\content_reviews.csv',encoding='utf-8', header=None)
scores = []

for x in df[0]:
    sen = x.lower()
    stopWords = set(stopwords.words('english'))
    words = word_tokenize(sen)
    
    wordsFiltered = []
    for w in words:
        if w not in stopWords:
            wordsFiltered.append(w)
    #print wordsFiltered
    posScore = 0
    negScore = 0
    for i in range(0,len(pos['positive'])):
        for s in wordsFiltered:
            if s in pos['positive'][i]['phrase']:
                posScore = posScore+pos['positive'][i]['value']
                
    for j in range(0,len(pos['negative'])):
        for s in wordsFiltered:
            if s in pos['negative'][j]['phrase']:
                negScore = negScore+pos['negative'][j]['value']
                
    if posScore > negScore:
        scores.append('positive')
        
    elif posScore < negScore:
        scores.append('negative')
    else:
        scores.append('neutral')
        
            
df['scores'] = scores
#df

df.to_csv('hotels_scores.csv', header=False, index=False,encoding='utf-8')

#After having csv containg reviews and their coresponding sentiment, pass them to ML model to train and get their accuracy

#loading file

def load_file():
    with open('C:\Users\user\Documents\sentiment-analysis-reviews\hotels_scores.csv') as csv_file:
        reader = csv.reader(csv_file,delimiter=",",quotechar='"')
        reader.next()
        data =[]
        target = []
        for row in reader:
            if row[0] and row[1]:
                data.append(row[0])
                target.append(row[1])

        return data,target

#creating the trem frequency matrix for the dataset
def preprocess():
    data,target = load_file()
    count_vectorizer = CountVectorizer(binary='true')
    data = count_vectorizer.fit_transform(data)
    tfidf_data = TfidfTransformer(use_idf=False).fit_transform(data)
    
    return tfidf_data

def learning_model(data,target):
    # preparing data for split validation. 60% training, 40% test
    data_train,data_test,target_train,target_test = cross_validation.train_test_split(data,target,test_size=0.3,random_state=43)
    classifier = BernoulliNB().fit(data_train,target_train)
    predicted = classifier.predict(data_test)
    evaluate_model(target_test,predicted)
    

def evaluate_model(target_true,target_predicted):
    print classification_report(target_true,target_predicted)
    print "The accuracy score is {:.2%}".format(accuracy_score(target_true,target_predicted))
        
def main():
    data,target = load_file()
    tf_idf = preprocess()
    learning_model(tf_idf,target)
# main()

# Some statistics about the reviews:
print("Number of reviews in reviews1.json = " + str(len(df.index)))
positive_rev = (df['scores'] == 'positive').sum()
negative_rev = (df['scores'] == 'negative').sum()
neutral_rev = (df['scores'] == 'neutral').sum()
print("Number of positive reviews =" + " "+str(positive_rev))
print("Number of negative reviews =" + " "+str(negative_rev))
print("Number of neutral reviews =" +" "+ str(neutral_rev))

df.describe()

print ('\nThe mean values are: ')

df_X = pd.read_json(file1,typ='series')
df_XX = json_normalize(df_X['Reviews'])
df_XX

Number of reviews in reviews1.json = 188
Number of positive reviews = 39
Number of negative reviews = 130
Number of neutral reviews = 19

The mean values are: 


,Author,AuthorLocation,Content,Date,"Ratings.Business service (e.g., internet access)",Ratings.Check in / front desk,Ratings.Cleanliness,Ratings.Location,Ratings.Overall,Ratings.Rooms,Ratings.Service,Ratings.Sleep Quality,Ratings.Value,ReviewID,Title
0,amacwow,Philadelphia,Hotel is clean and very close to airport. Staf...,"April 7, 2012",NaN,NaN,4,5,4.0,4,4,NaN,4,UR127417978,“Excellent location for business travelers sta...
1,robinhenshaw,"Rochdale, United Kingdom",Very convenient for LAX (free shuttle). New ve...,"April 5, 2012",NaN,NaN,4,3,3.0,3,3,4,3,UR127319807,“City Centre Holiday Inn”
2,zorro-plata,Dunedin,The thing I like most about Holiday Inn LAX is...,"April 2, 2012",NaN,NaN,4,5,4.0,4,4,5,4,UR127160327,"“Value, convenient, but ...”"
3,judystoks,"Porter, MN",We stayed here one night before taking a cruis...,"March 27, 2012",NaN,NaN,5,5,5.0,4,5,4,4,UR126728543,“Great One Night Stay”
4,Qwik16,"Simsbury, Connecticut",This hotel is just minutes away from LAX with ...,"March 23, 2012",NaN,NaN,5,5,3.0,4,4,3,5,UR126534324,“Close to Airport”
5,travelermanPhoenix,"Avondale, Arizona",I stay at this hotel a lot. The staff is nice ...,"March 22, 2012",NaN,NaN,3,2,3.0,3,5,3,3,UR126509591,“Airline Crew”
6,TXJones,"Dallas, TX",This hotel is good for a one night stay close ...,"March 15, 2012",NaN,NaN,3,5,3.0,3,4,NaN,3,UR126138691,“Good for a one night stay close to LAX”
7,Franklin J,"Houston, Texas",I've been trying to call the hotel for the pas...,"March 15, 2012",NaN,NaN,1,1,1.0,1,1,1,1,UR126136687,“horrible hotel”
8,winkrob92,,It's a hit &the miss one week go od another ba...,"March 2, 2012",NaN,NaN,3,2,3.0,3,4,NaN,3,UR125491474,“Airline Crew”
9,JubilantBeggar,"London, United Kingdom","Marginal hotel, but close in proximity to LAX....","February 26, 2012",NaN,NaN,4,5,3.0,4,4,3,5,UR125218028,"“Average hotel, but great for LAX flights”"
